In [ ]:
# Install required packages
!pip install -q evaluate rouge_score bert-score tqdm pandas

In [ ]:
import torch
from tqdm import tqdm
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import evaluate

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# Use current directory since the model files are in the current workspace
model_ckpt = "/kaggle/input/govreport-gelu/transformers/default/1"

# Load tokenizer and model 
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

In [ ]:
# Function to get model response
def get_response(input_text):
    """
    Generate a text summary using the fine-tuned PEGASUS-X model.
    """
    batch = tokenizer([input_text], truncation=True, padding='longest', max_length=12288, return_tensors="pt")
    gen_out = model_pegasus.generate(**batch, max_length=1024, num_beams=5, num_return_sequences=1, temperature=1.5)
    output_text = tokenizer.batch_decode(gen_out, skip_special_tokens=True)
    return output_text[0]

In [ ]:
# Data loading 
test = pd.read_parquet('/kaggle/input/govreport/test.parquet')

In [ ]:
# Generating predictions
pred = []
for i in tqdm(test['report'], total=len(test)):
    pred.append(get_response(i))

In [ ]:
ref_text = []
for i in tqdm(test['summary'], total=len(test)):
    ref_text.append(i)

## ROGUE

In [ ]:
rouge = evaluate.load("rouge")
results = rouge.compute(predictions=pred, references=ref_text)
print('ROUGE results:', results)

## BLEU

In [ ]:
bleu = evaluate.load("bleu")
bleu_results = bleu.compute(predictions=pred, references=[[ref] for ref in ref_text])
print("BLEU Results:")
print(bleu_results)
print(f"BLEU Score: {bleu_results['bleu']:.4f}")

## BERTScore

In [ ]:
bertscore = evaluate.load("bertscore")
bertscore_results = bertscore.compute(predictions=pred, references=ref_text, lang="en")
print("BERTScore Results:")
print(f"Precision: {sum(bertscore_results['precision'])/len(bertscore_results['precision']):.4f}")
print(f"Recall: {sum(bertscore_results['recall'])/len(bertscore_results['recall']):.4f}")
print(f"F1: {sum(bertscore_results['f1'])/len(bertscore_results['f1']):.4f}")